In [1]:
import os
from pprint import PrettyPrinter

import cv2 as cv
import numpy as np

pp = PrettyPrinter(indent=2, width=80)

# Edge detection

In [2]:
test_dir = 'test_data'
sample_base = 'test'
sample_img_file = os.path.join(test_dir, f'{sample_base}.jpg')

img = cv.imread(sample_img_file)

# Find contours

`contours, hierarchy = cv.findContours(source_image, retrieval_mode, approximation_method)`

In [3]:
imgray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
ret_grey, thresh_grey = cv.threshold(imgray, 127, 255, 0)

In [4]:
contours, hierarchy = cv.findContours(thresh_grey, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

# Draw the found contours on the image

In [5]:
nth_contour = -1
line_color = (0, 255, 0)
thickness = 3

In [ ]:
draw = cv.drawContours(img, contours, nth_contour, line_color, thickness)

In [115]:
cv.imshow('contours', draw)
cv.waitKey(0)
cv.destroyAllWindows()

contour 중에서 글자를 포함하고 있는 것을 찾아야 한다.
 - condition: IoU(contour, characterPoly)  == area(characterPoly)
 - 순서:
    1. 모든 `field`마다 소속 contours를 매긴다
        (혹은 모든 contour마다 어떤 `field`가 들어가는지 매긴다)
    2. field가 없는 contours를 지운다.
    3.
contour c1, c2는 둘 다 글자를 포함하고 있다. c1은 c2에 완전히 포함된다. 그러면 c2

In [6]:
from collections import defaultdict

from shapely.geometry import Polygon
import parsing

In [7]:
# Get the individual contour from `contours`.
contours_valid = tuple(contour for contour in contours if len(contour) > 3)

In [8]:
contours_valid_squeezed = [np.squeeze(contour, axis=1)
                           for contour in contours_valid]

In [9]:
contours_hashable = [
    tuple(tuple(point) for point in contour)
    for contour in contours_valid_squeezed
]


In [10]:
# Polygon에 맞는 형식으로 각 필드 변환.
sample_ocr_file = f'test_data/{sample_base}.json'
ocr_fields = parsing.parse_result(sample_ocr_file)

for field in ocr_fields:
    vertices = field['boundingPoly']['vertices']  # [좌상, 우상, 우하, 좌하]
    inferText = field['inferText']
    inferConfidence = field['inferConfidence']

    field['boundingPoly'] = parsing.simplify_vertices(vertices)

In [11]:
print(ocr_fields[0])
print(contours_hashable[0])

{'valueType': 'ALL', 'boundingPoly': ((89.0, 85.0), (140.0, 85.0), (140.0, 115.0), (89.0, 115.0)), 'inferText': '내부', 'inferConfidence': 1.0, 'type': 'NORMAL', 'lineBreak': False}
((0, 0), (0, 346), (500, 346), (500, 0))


In [12]:
in_contours = defaultdict(list)
having_field = defaultdict(list)

for fidx, field in enumerate(ocr_fields):
    bounding_poly = Polygon(field['boundingPoly'])
    for cidx, contour in enumerate(contours_hashable):
        contour_poly = Polygon(contour)

        if bounding_poly.intersection(contour_poly).area >= bounding_poly.area:
            in_contours[fidx].append(contour)
            having_field[contour].append(field)

# 글자가 있는 영역만 필터링 됐는지 시각화

In [13]:
for contour in list(having_field.keys()):
    contour = np.array([[list(point)] for point in contour])
    break

In [21]:
contours_with_character = [np.array([[list(point)] for point in contour])
                           for contour in list(having_field.keys())]


In [22]:
len(contours_with_character)

3

In [23]:
draw = cv.drawContours(img, contours_with_character, -1, line_color, thickness)
cv.imshow('contours only have characters in', draw)
cv.waitKey(0)
cv.destroyAllWindows()

# 사각형 모양이 아닌 영역은 단순 도형으로 근사하기

In [ ]:
for cnt in contours:
    epsilon = 0.02 * cv.arcLength(cnt, True)
    approx = cv.approxPolyDP(cnt, epsilon, True)
    print(len(approx))

draw = cv.drawContours(img, [approx], 0, (0, 255, 255), 5)